In [9]:
import json
import numpy as np
import pandas as pd
import os

In [14]:
contains_bills = []

for i in range(1, 114):
    path = '/media/anne/LACIE SHARE/DataScienceFinalProject/' + str(i) + '/bills'  #this path needs to be changed
    if os.path.exists(path):
        contains_bills.append(i)
        
print contains_bills

[6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]


In [15]:
def find_subject_in_bills(subject):
    res = []
    for i in range(0, len(tmp)):
        if type(tmp[i]) != dict:
            res.append(0)
        else:
            if subject in tmp[i]['subjects']:
                res.append(1)
            else:
                res.append(0)
    return res

def get_subjects(isAmendment, title):
    if type(title) != str: #
        tmp.append(np.nan)
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
        else:
            bill = title
        path = "/media/anne/LACIE SHARE/DataScienceFinalProject/" + str(congress_no) + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/" + bill + "/data.json"
        if os.path.exists(path):
            f = open(path, 'r')
            vote = json.loads(f.read(), 'utf-8')
            obj = {}
            if 'subjects' in vote:
                obj['subjects'] = vote['subjects']
            else:
                obj['subjects'] = np.nan
            if len(vote['committees']):
                obj['committee'] = vote['committees'][0]['committee']
            else:
                obj['committee'] = np.nan
            if 'short_title' in vote:
                obj['billTitle'] = vote['short_title']
            else:
                obj['billTitle'] = np.nan
            if 'official_title' in vote:
                obj['officialTitle'] = vote['official_title']
            else:
                obj['officialTitle'] = np.nan
            if 'sponsor' in vote and vote['sponsor']:
                name = vote['sponsor']['name'].split(', ')
                filtered_name = name[1] + "#" + name[0] + "#" + vote['sponsor']['state']
                obj['sponsor'] = filtered_name
            else:
                obj['sponsor'] = np.nan
            tmp.append(obj)            
        else:
            tmp.append(np.nan)

In [22]:
# for congress_no in contains_bills:
for congress_no in range(1,114):
    print congress_no
    for body in ['house', 'senate']:
        tmp = []
        path = 'csv/' + str(congress_no) + body + '.csv'   #this path needs to be changed
        path2 = 'cleanedcsv/' + str(congress_no) + body + '.csv'  #this path needs to be changed
        df = pd.read_csv(path)
        subjects_dict = {'committee': [], 'billTitle': [], 'sponsor': [], 'officialTitle': []}
        for i in range(0, len(df.title)):
            get_subjects(df.isAmendment[i], df.title[i])
            
        count_subjects_dict = {}

        for i in range(0, len(tmp)):
            if type(tmp[i]) == dict:
                subjects_dict['committee'].append(str(tmp[i]['committee']))
                subjects_dict['billTitle'].append(str(tmp[i]['billTitle']))
#                 print tmp[i]['officialTitle'].encode('utf-8')
                subjects_dict['officialTitle'].append(tmp[i]['officialTitle'].encode('utf-8'))
                subjects_dict['sponsor'].append(str(tmp[i]['sponsor']))
                if type(tmp[i]['subjects']) == list:        
                    for j in range(0, len(tmp[i]['subjects'])):
                        if tmp[i]['subjects'][j] in count_subjects_dict:
                            count_subjects_dict[tmp[i]['subjects'][j]] += 1
                        else:
                            count_subjects_dict[tmp[i]['subjects'][j]] = 1
            else:
                subjects_dict['committee'].append(np.nan)
                subjects_dict['billTitle'].append(np.nan)
                subjects_dict['officialTitle'].append(np.nan)
                subjects_dict['sponsor'].append(np.nan)

        for key in count_subjects_dict:
            subjects_dict[key] = find_subject_in_bills(key)
        
        subjects_df = pd.DataFrame(data=subjects_dict)

        final = pd.concat([df, subjects_df], axis=1)
        final.to_csv(path2, index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [11]:
for key in subjects_dict:
    if len(subjects_dict[key]) != 697:
        print key

billTitle
committee
